<a href="https://colab.research.google.com/github/thienan95/assignment/blob/master/Assignment_3_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [0]:
# Import pandas, numpy
import pandas as pd
import numpy as np


In [0]:
# Read dataset with extra params sep='\t', encoding="latin-1"
path = '/content/labeledTrainData.tsv'
sentiment = pd.read_csv(path,sep='\t', encoding = 'latin-1')
sentiment.sample(20)

,id,sentiment,review
18347,6307_8,1,I had never heard of this flick despite the co...
21598,9371_7,1,One thing I always liked about Robert Ludlum t...
11378,1215_10,1,This is my favorite of the three care bears mo...
553,11950_8,1,I went into this movie knowing nothing about i...
10567,5722_1,0,First I'd like to excuse my bad English.<br />...
684,11327_4,0,"Set in a post-apocalyptic environment, cyborgs..."
7858,8045_8,1,I finally got myself set up on mail order DVD ...
13766,7512_1,0,I had to walk out of the theater. After an hou...
2808,265_1,0,The plot of this terrible film is so convolute...
17790,9615_1,0,It seems evident from this adaptation that he ...


## 2. Preprocessing

In [0]:
# stop words
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text) # Your code here
    
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Your code here
    
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [0]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
porter = PorterStemmer()

def tokenizer_porter(text):
  token = [] 
  token = text.split()
  return token
def tokenizer_porter(text):
    token = []
    token = [porter.stem(word) for word in text.split()]
    return token 

In [0]:
# split the dataset in train and test
from sklearn.model_selection import train_test_split
X = sentiment['review']
y = sentiment['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X,y)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [0]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
tfidf_vectorizer = TfidfVectorizer(
    analyzer='word',
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None)

lr = LogisticRegression()
clf = Pipeline([('tfidf', tfidf_vectorizer), ('lr', lr)])

clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function <lambda> at 0x7fb979d79bf8>,
                                 smooth_idf=True, stop_words=None,
                                 strip_accents=None, subl...
                                 token_pattern=None,
                                 tokenizer=<function <lambda> at 0x7fb979d79d90>,
                                 use_idf=True, vocabulary=None)),
                ('lr',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                   

## 4. Evaluate Model

In [0]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
y_predict = clf.predict(X_test)

## 5. Export Model 

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import multilabel_confusion_matrix
accuracy_score(y_test, y_predict)

0.64736

In [0]:
classification_report(y_test, y_predict).split('\n')

['              precision    recall  f1-score   support',
 '',
 '           0       0.65      0.63      0.64      3136',
 '           1       0.64      0.67      0.65      3114',
 '',
 '    accuracy                           0.65      6250',
 '   macro avg       0.65      0.65      0.65      6250',
 'weighted avg       0.65      0.65      0.65      6250',
 '']

In [0]:
# Using pickle to export our trained model
import pickle
import os


PicklingError: ignored